# Ontology interface

This tutorial demonstrates basic usages of __owlready2__ API for ontology manipulation. Notably, new ontology concept triple classes (subject, predicate, object) will be dynamically created, with optional existing ontology parent classes that are loaded from an OWL ontology. Then through the interconnected relations specified in triples, designators and their corresponding ontology concepts can be double-way queried for certain purposes, eg. making a robot motion plan. 

In [ ]:
from pathlib import Path 
from typing import Optional, List, Type
import pycram
from pycram.designator import DesignatorDescription, ObjectDesignatorDescription

# Owlready2

[Owlready2](https://owlready2.readthedocs.io/en/latest/intro.html) is a Python package providing a transparent access to OWL ontologies. It supports various manipulation operations, including but not limited to loading, modification, saving ontologies. Built-in supported reasoners include [HermiT](http://www.hermit-reasoner.com) and [Pellet](https://github.com/stardog-union/pellet).

In [ ]:
from owlready2 import *

# Ontology Manager

`OntologyManager` is the singleton class acting as the main interface between PyCram with ontologies, whereby object instances in the former could query relevant information based on the semantic connection with their corresponding ontology concepts.

Such connection, as represented by triples (subject-predicate-object), could be also created on the fly if not pre-existing in the loaded ontology.

Also new and updated concepts with their properties defined in runtime could be stored into an [SQLite3 file database](https://owlready2.readthedocs.io/en/latest/world.html) for reuse.

Here we will use [SOMA ontology](https://ease-crc.github.io/soma) as the baseline to utilize the generalized concepts provided by it.

In [ ]:
from pycram.ontology import OntologyManager, SOMA_HOME_ONTOLOGY

OntologyManager(SOMA_HOME_ONTOLOGY)
onto = OntologyManager.onto
soma = OntologyManager.soma
dul = OntologyManager.dul

## Ontology Concept class
A built-in class named __`OntologyConcept`__, inheriting from __`owlready2.Thing`__, is created when `OntologyManager` is initialized, as the super class for all dynamically made ontology classes later on. It is then accessed by __`OntologyManager.onto.OntologyConcept`__

Notable members:
- `designators`: a list of `DesignatorDescription` instances associated with the ontology concept
- `resolve`: a `Callable` returning a list of `DesignatorDescription`. It is used to provide which specific designators inferred from the ontology concept. Most typically, they are `designators`, but can be only a subset of it given certain conditions.  

## Query ontology classes and their properties

Classes in the loaded ontology can be queried based on their exact names, or part of them, or by namespace.

By specifying `print_info` parameter as `True`, essential info (ancestors, super/sub-classes, properties, direct instances, etc.) of the found ontology class will be printed.

In [ ]:
onto_designed_container_class = OntologyManager.get_ontology_class('DesignedContainer', print_info=True)
OntologyManager.print_ontology_class(onto_designed_container_class)
OntologyManager.get_ontology_classes_by_subname('PhysicalObject', print_info=True)
OntologyManager.get_ontology_classes_by_namespace('SOMA')

__Descendants__ of an ontology class can be also queried by

In [ ]:
OntologyManager.get_ontology_descendant_classes(onto_designed_container_class)

## Create a new ontology class and its individual

A new ontology class can be created dynamically as inheriting from an existing class in the loaded ontology.
Here we create the class and its instance, also known as [__individual__](https://owlready2.readthedocs.io/en/latest/class.html#creating-equivalent-classes) in ontology terms.

In [ ]:
onto_custom_container_class = OntologyManager.create_ontology_concept_class('CustomContainerConcept',
                                                                            onto_designed_container_class)
custom_container_concept = onto_custom_container_class('onto_custom_container_concept')

## Access ontology classes and individuals
All ontology classes created on the fly inherit from __`owlready2.Thing`__, and so share the same namespace with the loaded ontology instance `onto`. They can then be accessible through that namespace by __`onto.<class_name>`__.
The same applies for individuals of those classes, accessible by __`onto.<class_individual_name>`__

In [ ]:
OntologyManager.print_ontology_class(onto.OntologyConcept)
OntologyManager.print_ontology_class(onto.CustomContainerConcept)
print(f"custom_container_concept is {onto.onto_custom_container_concept}: {custom_container_concept is onto.onto_custom_container_concept}")

For ones already existing in the ontology, they can only be accessed through their corresponding ontology, eg: `soma` as follows

In [ ]:
OntologyManager.print_ontology_class(soma.Cup)

## Connect ontology class individuals with designators
After creating `custom_container_concept` class, we connect it to a designator (say `obj_designator`) by:
- Append to `obj_designator.onto_concepts` with `custom_container_concept`
- Append to `custom_container_concept.designators` with `obj_designator`

In [ ]:
custom_container_designator = ObjectDesignatorDescription(names=["obj"])
custom_container_designator.onto_concepts.append(custom_container_concept)
custom_container_concept.designators.append(custom_container_designator)

We can also automatize all the above setup with a single function call

In [ ]:
another_custom_container_designator = OntologyManager.create_ontology_linked_designator(designator_name="another_custom_container",
                                                                                        designator_class=ObjectDesignatorDescription,
                                                                                        onto_concept_name="AnotherCustomContainerConcept",
                                                                                        onto_parent_class=onto_designed_container_class)
print(another_custom_container_designator.onto_concepts)
print(onto.AnotherCustomContainerConcept.instances()[0].get_default_designator().names)

## Create new ontology triple classes

Concept classes of a triple, aka [__subject, predicate, object__], can be created dynamically. Here we will make an example creating ones for [__handheld objects__] and [__placeholder objects__], with a pair of predicate and inverse predicate signifying their mutual relation.

In [ ]:
PLACEABLE_ON_PREDICATE_NAME = "placeable_on"
HOLD_OBJ_PREDICATE_NAME = "hold_obj"
OntologyManager.create_ontology_triple_classes(onto_subject_parent_class=soma.DesignedContainer,
                                               subject_class_name="OntologyPlaceHolderObject",
                                               onto_object_parent_class=soma.Shape,
                                               object_class_name="OntologyHandheldObject",
                                               predicate_name=PLACEABLE_ON_PREDICATE_NAME,
                                               inverse_predicate_name=HOLD_OBJ_PREDICATE_NAME,
                                               onto_property_parent_class=soma.affordsBearer,
                                               onto_inverse_property_parent_class=soma.isBearerAffordedBy)

There, we use `soma.DesignedContainer` & `soma.Shape`, existing concept in SOMA ontology, as the parent classes for the subject & object concepts respectively.
There is also a note that those classes, as inheriting from ##owlready2##-provided classes, are automatically given the namespace `onto`, so later on to be accessible through it.

Then now we define some instances of the newly created triple classes, and link them to object designators, again using __`OntologyManager.create_ontology_linked_designator()`__

In [ ]:
def create_ontology_handheld_object(obj_name: str, onto_parent_class: Type[Thing]):
    return OntologyManager.create_ontology_linked_designator(designator_name=obj_name,
                                                             designator_class=ObjectDesignatorDescription,
                                                             onto_concept_name=f"Onto{obj_name}",
                                                             onto_parent_class=onto.OntologyHandheldObject)
# Holdable Objects
cookie_box = create_ontology_handheld_object("cookie_box", onto.OntologyHandheldObject)
egg = create_ontology_handheld_object("egg", onto.OntologyHandheldObject)
    
# Placeholder objects
placeholders = [create_ontology_handheld_object(obj_name, onto.OntologyPlaceHolderObject)
                for obj_name in ['table', 'stool', 'shelf']]

egg_tray = create_ontology_handheld_object("egg_tray", onto.OntologyPlaceHolderObject)

### Create ontology relations

Now we will create ontology relations or predicates between __placeholder objects__ and __handheld objects__ with __`OntologyManager.set_ontology_relation()`__

In [ ]:
for place_holder in placeholders:
    OntologyManager.set_ontology_relation(subject_designator=cookie_box, object_designator=place_holder,
                                          predicate_name=PLACEABLE_ON_PREDICATE_NAME)

OntologyManager.set_ontology_relation(subject_designator=egg_tray, object_designator=egg,
                                      predicate_name=HOLD_OBJ_PREDICATE_NAME)

## Query designators based on their ontology-concept relations

Now we can make queries for designators from designators, based on the relation among their corresponding ontology concepts setup above

In [ ]:
print(f"{cookie_box.names}'s placeholder candidates:",
      f"""{[placeholder.names for placeholder in
            OntologyManager.get_designators_by_subject_predicate(subject=cookie_box,
                                                                 predicate_name=PLACEABLE_ON_PREDICATE_NAME)]}""")

print(f"{egg.names}'s placeholder candidates:",
      f"""{[placeholder.names for placeholder in
            OntologyManager.get_designators_by_subject_predicate(subject=egg,
                                                                 predicate_name=PLACEABLE_ON_PREDICATE_NAME)]}""")

for place_holder in placeholders:
    print(f"{place_holder.names} can hold:",
          f"""{[placeholder.names for placeholder in
                OntologyManager.get_designators_by_subject_predicate(subject=place_holder,
                                                                     predicate_name=HOLD_OBJ_PREDICATE_NAME)]}""")

print(f"{egg_tray.names} can hold:",
      f"""{[placeholder.names for placeholder in
            OntologyManager.get_designators_by_subject_predicate(subject=egg_tray,
                                                                 predicate_name=HOLD_OBJ_PREDICATE_NAME)]}""")

# Practical examples

## Example 1
How about creating ontology concept classes encapsulating `pycram.enums.ObjectType`? We can do it by:

In [ ]:
from pycram.enums import ObjectType

# Create a generic ontology concept class for edible objects
generic_edible_class = OntologyManager.create_ontology_concept_class('GenericEdible')

# Create a list of object designators sharing the same concept class as [generic_edible_class]
edible_obj_types = [ObjectType.MILK, ObjectType.BREAKFAST_CEREAL]
for object_type in ObjectType:
    if object_type in edible_obj_types:
        # Create a designator for the edible object
        OntologyManager.create_ontology_object_designator_from_type(object_type, generic_edible_class)

print(f'{generic_edible_class.name} object types:')
for edible_onto_concept in generic_edible_class.direct_instances():
    print(edible_onto_concept, [des.types for des in edible_onto_concept.designators])


## Example 2
We could also make use of relations between ontology concepts that designators are associated with, to enable more abstract inputs in robot motion plan.

In a similar style to the scenario of __placeholder objects__ and __handheld objects__ above, but with a little bit difference, we will ask the robot to query which content holders (eg. cup, pitcher, bowl) whereby a milk box could be pourable into.

Basically, we will provide an ontology-based implementation for the query:
 
`abstract_ontology_concept -> specific_objects_in_world?`

To achieve it, we will create triple classes and configure a customized `resolve()` for the abstract concept, which returns its associated specific designators.
These designators are then used to again resolve for the target objects of interest, which become the inputs to a robot motion plan.

### Setup simulated environment

In [ ]:
from pycram.bullet_world import BulletWorld, Object
from pycram.enums import ObjectType
from pycram.pose import Pose

from pycram.process_module import simulated_robot
from pycram.designators.action_designator import *
from pycram.designators.location_designator import *

world = BulletWorld()
plane = BulletWorld.current_bullet_world.objects[0]
kitchen = Object("kitchen", ObjectType.ENVIRONMENT, "kitchen.urdf")
pr2 = Object("pr2", ObjectType.ROBOT, "pr2.urdf")
kitchen_desig = ObjectDesignatorDescription(names=["kitchen"])
robot_desig = ObjectDesignatorDescription(names=["pr2"]).resolve()

### Create PourableObject-LiquidHolder triple ontology classes

In [ ]:
POURABLE_INTO_PREDICATE_NAME = "pourable_into"
HOLD_LIQUID_PREDICATE_NAME = "hold_liquid"
OntologyManager.create_ontology_triple_classes(onto_subject_parent_class=soma.DesignedContainer,
                                               subject_class_name="OntologyLiquidHolderObject",
                                               onto_object_parent_class=soma.Shape,
                                               object_class_name="OntologyPourableObject",
                                               predicate_name=POURABLE_INTO_PREDICATE_NAME,
                                               inverse_predicate_name=HOLD_LIQUID_PREDICATE_NAME,
                                               onto_property_parent_class=soma.affordsBearer,
                                               onto_inverse_property_parent_class=onto.HOLD_OBJ_PREDICATE_NAME)

### Spawn a pourable object & liquid holders into the world and Create their designators

In [ ]:
# Holdable obj
milk_box = Object("milk_box", ObjectType.MILK, "milk.stl")
milk_box_designator = create_ontology_handheld_object(milk_box.name, onto.OntologyPourableObject)

# Liquid-holders
cup = Object("cup", ObjectType.JEROEN_CUP, "jeroen_cup.stl", pose=Pose([1.4, 1, 0.9]))
bowl = Object("bowl", ObjectType.BOWL, "bowl.stl", pose=Pose([1.4, 0.5, 0.9]))
pitcher = Object("pitcher", ObjectType.GENERIC_OBJECT, "Static_MilkPitcher.stl", pose=Pose([1.4, 0, 0.9]))
milk_holders = [cup, bowl, pitcher]
milk_holder_designators = [create_ontology_handheld_object(obj.name, onto.OntologyLiquidHolderObject)
                           for obj in milk_holders]

### Create an ontology relation between the designators of the pourable object & its liquid holders

In [ ]:
for milk_holder_desig in milk_holder_designators:
    OntologyManager.set_ontology_relation(subject_designator=milk_box_designator, object_designator=milk_holder_desig,
                                          predicate_name=POURABLE_INTO_PREDICATE_NAME)

### Set up `resolve` for the ontology concept of the pourable object

In [ ]:
milk_box_concept = milk_box_designator.get_default_onto_concept()
def milk_box_concept_resolve(): 
    object_designator = OntologyManager.get_designators_by_subject_predicate(subject=milk_box_designator, predicate_name=POURABLE_INTO_PREDICATE_NAME)[0]
    return object_designator, object_designator.resolve()

milk_box_concept.resolve = milk_box_concept_resolve

Here, for demonstration purpose only, we specify the resolving result by __`milk_box_concept`__ as __`cup`__, the first-registered (default) pourable-into target milk holder, utilizing the ontology relation setup above.

Now, we can query the milk box's target liquid holder by resolving `milk_box_concept`

In [ ]:
target_milk_holder_designator, target_milk_holder = milk_box_concept.resolve()
print('Pickup target object:', target_milk_holder.name)

### Robot picks up the target liquid holder

In [ ]:
with simulated_robot:
    ParkArmsAction([Arms.BOTH]).resolve().perform()

    MoveTorsoAction([0.3]).resolve().perform()

    pickup_pose = CostmapLocation(target=target_milk_holder, reachable_for=robot_desig).resolve()
    pickup_arm = pickup_pose.reachable_arms[0]

    print(pickup_pose, pickup_arm)

    NavigateAction(target_locations=[pickup_pose.pose]).resolve().perform()

    PickUpAction(object_designator_description=target_milk_holder_designator, arms=[pickup_arm], grasps=["front"]).resolve().perform()

    ParkArmsAction([Arms.BOTH]).resolve().perform()

    place_island = SemanticCostmapLocation("kitchen_island_surface", kitchen_desig.resolve(), target_milk_holder_designator.resolve()).resolve()

    place_stand = CostmapLocation(place_island.pose, reachable_for=robot_desig, reachable_arm=pickup_arm).resolve()

    NavigateAction(target_locations=[place_stand.pose]).resolve().perform()

    PlaceAction(target_milk_holder_designator, target_locations=[place_island.pose], arms=[pickup_arm]).resolve().perform()

    ParkArmsAction([Arms.BOTH]).resolve().perform()

world.exit()